# Database CRUD

David Scanu 

---
Contexte du projet
Futurs développeur en intelligence artificielle, vous serez amenés la plupart du temps à récupérer, stocker puis analyser des données. Connaître le fonctionnelemment des bases de données relationnelles est donc primordial...


NB: Vous aurez accès au notebook "infos utiles.ipynb" sur teams pour vous aider

---

Votre mission si vous l'acceptez :

## Dans phpMyAdmin *

I ) Créer une base de données que vous appelerez : CountriesDatas II ) Vous ajouterez deux tables à de cette nouvelle base:
[* les colonnes sont notées ci-dessous entre paranthèses]

* a) countries (code, name)
* b) populations (id, LOCATION, TIME , Value)

NB: Pour l'id vous pourrez l'ajouter lorsque vous ferez les INSERTs, par exemple le 1er INSERT possède un         id = 1, le 2ème id = 2, etc _
​
## Dans un jupyter notebook

III ) Ensuite, vous devrez peupler les tables countries et populations respectivement à l'aide des fichiers suivants : country.txt et population.csv (voir pieces jointes)

IV ) Enfin vous executerez une query de type SELECT sur la table population, en incluant un JOIN sur la table countries. Le but étant d'afficher les elements les "name" de la table countries au lieu des "LOCATION" de la table populations.

NB: Votre query devrait vous retourner une liste de tuple du type :

[(217, 'Canada', datetime.date(1950, 1, 1), 14018600.0), (218, 'Canada', datetime.date(1951, 1, 1), 14318200.0), (219, 'Canada', datetime.date(1952, 1, 1), 14776300.0), (220, 'Canada', datetime.date(1953, 1, 1), 15169800.0)]

V ) Pour finir, vous convertirez le resultat de la query précedente en dataframe

In [35]:
import mysql.connector
from mysql.connector import errorcode
import json
import pandas as pd

## Connection à la DB

In [66]:
# CONSTANTS
VERBOSE = False
CONFIG_FILE_PATH = 'config.json'

# Configuration file for our app
def get_db_config(config_file_path):
    """
    Function that gets database ocnfiguration from a JSON file.
    """
    # Opening JSON file
    f = open(config_file_path, 'r')
    # returns JSON object as a dictionary
    config = json.load(f)
    # Closing file
    f.close()

    return config

# Database connection
def db_connect(config, VERBOSE=True):
    """
    Function that connects to the DB
    """
    bdd = None

    if VERBOSE: 
        print("Lancement de MYSQL...")

    try:
        bdd = mysql.connector.connect(**config)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)
            print ("Error code: ", err.errno)        # error number
            print ("SQLSTATE value: ", err.sqlstate) # SQLSTATE value
            print ("Error message: ", err.msg)       # error message
            print ("Error: ", str(err))              # errno, sqlstate, msg values 

    if bdd.is_connected():

        if VERBOSE:
            bdd_info = bdd.get_server_info()
            print(f"Connexion à {bdd_info} OK")

    return bdd

# Close DB connection
def db_close(bdd, VERBOSE=True):
    """
    Function that closes the database connection.
    """
    bdd.close()
    if not bdd.is_connected() and VERBOSE :
        print("Fermeture de la connexion.")

In [67]:
# Database ocnfiguration
CONFIG = get_db_config(CONFIG_FILE_PATH)
# Database connection
bdd = db_connect(CONFIG)

Lancement de MYSQL...
Connexion à 8.0.31 OK


## Sauvegarde des fichiers dans la DB

Ensuite, vous devrez peupler les tables countries et populations respectivement à l'aide des fichiers suivants : country.txt et population.csv (voir pieces jointes)

In [68]:
# Read country.txt file
def read_file_txt(file_path):

    """
    Enables to read a .txt file and to extract its informations.
    the function return a list of dictionaries.
    """
    f = open(f"{file_path}", "r")
    
    # Ce dictionnaire contiendra tous les "country_code" et "capital"
    file_content = []
    
    for line in f:
        # la ligne ci-dessous permet de ne pas recupérer les titres
        # qui ne nous intéressent pas.
        if not False :
            # On converti ensuite chaque ligne, de type string (par exemple "FR Paris"),
            # en tableau (içi appelé "split_line") grâce à la fonction/ méthode "split()"
            # Dans le cas de la ligne "FR Paris", split_line[0] = "FR" 
            # et split_line[1] = "Paris"         
            line_dict = {}
            line_dict['country_code'] = line.split('__****__')[0].strip()
            line_dict['country_name'] = line.split('__****__')[1].strip('\n').strip()
            # print(line_dict)
            file_content.append(line_dict)
        
    return file_content

countries_list = read_file_txt('data/country.txt')
# print(countries_list)

In [69]:
# Insert single row into the DB
def insert_into_row(bdd, table, pk, data_dict, VERBOSE=True):
    """
    Function that saves row into DB. Values must be passed as a dictionnary : 
    
    data_dict = {
        'col_name_1' : 'col_value_1',
        'col_name_2' : 'col_value_2'
    }   
    """

    # Try this code
    try:
        # open cursor
        cursor = bdd.cursor()
        
        #  Adding columns to query
        col_names_str = ""
        for key in data_dict.keys():
            col_names_str += key + ', '
        col_names_str = col_names_str.strip(', ')
        # Query
        query = f"""INSERT IGNORE INTO {table} ({col_names_str}) """

        # Adding values to query
        values_str = ""
        for value in data_dict.values():
            values_str += f'"{value}", '
        values_str = values_str.strip(', ')
        # query
        query += f"""VALUES ({values_str})"""

        # On duplicate
        key_value_str = ""
        for key, value in data_dict.items():
            if not key == pk:
                key_value_str += f"""{key} = "{value}", """
        key_value_str = key_value_str.strip(', ')
        query += f""" ON DUPLICATE KEY UPDATE {key_value_str};"""

        # print("The query =>", query)

        #Cursor Execute
        cursor.execute(query)
        bdd.commit()
        return True

    # If Error, print the Error
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)
            print ("Error code: ", err.errno)        # error number
            print ("SQLSTATE value: ", err.sqlstate) # SQLSTATE value
            print ("Error message: ", err.msg)       # error message
            print ("Error: ", str(err))              # errno, sqlstate, msg values 


    finally: # Excecute no matter what happened before 
        if bdd.is_connected():
            # Close curor
            cursor.close()

In [70]:
# Insert list of dictionnary into DB
# Saving countries into DB
def save_countries(bdd, file_list, VERBOSE=True):
    for i, row in enumerate(file_list):
        row['country_id'] = i
        insert_into_row(bdd, "countries", "country_id", row, VERBOSE)

save_countries(bdd, countries_list)

In [72]:
# Read population.csv file
def read_pop_file_csv(file_path):

    """
    Enables to read a .txt file and to extract its informations.
    the function return a Dictionary.
    """
    f = open(f"{file_path}", "r")
    
    # Ce dictionnaire contiendra tous les "country_code" et "capital"
    file_content = []
    
    for line in f:
        # la ligne ci-dessous permet de ne pas recupérer les titres
        # qui ne nous intéressent pas.
        if not "LOCATION" in line :
            # On converti ensuite chaque ligne, de type string (par exemple "FR Paris"),
            # en tableau (içi appelé "split_line") grâce à la fonction/ méthode "split()"
            # Dans le cas de la ligne "FR Paris", split_line[0] = "FR" 
            # et split_line[1] = "Paris"       		
            line_dict = {}
            line_dict['pop_location'] = line.split(',')[0].strip('"')
            line_dict['pop_time'] = line.split(',')[5].strip('"')
            line_dict['pop_value'] = line.split(',')[6]
            file_content.append(line_dict)
        
    return file_content

populations_list = read_pop_file_csv('data/population.csv')
# print(populations_list)

In [73]:
# Saving population.csv to 'populations' table.
def save_populations(bdd, file_list):

    for i, row in enumerate(file_list):

        pop_id = i
        pop_location = row['pop_location']
        pop_time = row['pop_time']
        pop_value = row['pop_value']

        try:
            # open cursor
            cursor = bdd.cursor()

            query = f"""INSERT IGNORE INTO populations (pop_id, pop_location, pop_time, pop_value)
            VALUES ('{pop_id}', '{pop_location}', '{pop_time}', '{pop_value}');"""
            
            # print("The query =>", query)

            #Cursor Execute
            cursor.execute(query)
            bdd.commit()

        # If Error, print the Error
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Something is wrong with your user name or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Database does not exist")
            else:
                print(err)
                print ("Error code: ", err.errno)        # error number
                print ("SQLSTATE value: ", err.sqlstate) # SQLSTATE value
                print ("Error message: ", err.msg)       # error message
                print ("Error: ", str(err))              # errno, sqlstate, msg values 

        finally: # Excecute no matter what happened before 
            if bdd.is_connected():
                # Close curor
                cursor.close()

save_populations(bdd, populations_list)

## Afficher

IV ) Enfin vous executerez une query de type **SELECT** sur la table population, en incluant un **JOIN** sur la table countries. Le but étant d'afficher les elements les "name" de la table countries au lieu des "LOCATION" de la table populations.

NB: Votre query devrait vous retourner une liste de tuple du type :

[(217, 'Canada', datetime.date(1950, 1, 1), 14018600.0), (218, 'Canada', datetime.date(1951, 1, 1), 14318200.0), (219, 'Canada', datetime.date(1952, 1, 1), 14776300.0), (220, 'Canada', datetime.date(1953, 1, 1), 15169800.0)]

In [75]:
# SELECT populations
def get_populations(bdd):

    try:
        # open cursor
        cursor = bdd.cursor()

        query = f"""SELECT pop_id, country_name, pop_time, pop_value FROM populations JOIN countries ON pop_location=country_code"""
        
        # print("The query =>", query)

        #Cursor Execute
        cursor.execute(query)
        result_rows = cursor.fetchall()
        col_names = cursor.column_names
        count_cols = len(cursor.description)
        count_rows = cursor.rowcount
        
        print(f"Query is => {query}")
        print(f"Displaying {count_rows} rows and {count_cols} columns")
        print(f"Columns are : {col_names}.")

        # check if DB is empty
        if count_rows == 0 : 
            return "There are no countries yet!"
        else : 
            return result_rows, col_names

    # If Error, print the Error
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)
            print ("Error code: ", err.errno)        # error number
            print ("SQLSTATE value: ", err.sqlstate) # SQLSTATE value
            print ("Error message: ", err.msg)       # error message
            print ("Error: ", str(err))              # errno, sqlstate, msg values 

    finally: # Excecute no matter what happened before 
        if bdd.is_connected():
            # Close curor
            cursor.close()



In [76]:
countries_rows, countries_col_names = get_populations(bdd)

Query is => SELECT pop_id, country_name, pop_time, pop_value FROM populations JOIN countries ON pop_location=country_code
Displaying 12337 rows and 4 columns
Columns are : ('pop_id', 'country_name', 'pop_time', 'pop_value').


### Convertir en DataFrame

V ) Pour finir, vous convertirez le resultat de la query précedente en dataframe

In [79]:
def convert_df(rows, col_names):
    result_df = pd.DataFrame(rows, columns=col_names)
    return result_df

df = convert_df(countries_rows, countries_col_names)
df.rename(columns = {'pop_id':'ID', 'country_name' : 'Country Name', 'pop_time' : 'Year', 'pop_value' : 'Value'}, inplace = True)

In [80]:
df.head()

,ID,Country Name,Year,Value
0,0,Australia,1951,2.97113233154413
1,1,Australia,1952,2.55055392616693
2,2,Australia,1953,2.07028310079315
3,3,Australia,1954,1.94207797806087
4,4,Australia,1955,2.37244756022923
